In [30]:
import torch as th

class RestrictedTorchTensor(th.Tensor):
    """A tensor class which returns a NotImplementedError for all methods which you do not
    explicitly override."""

    def init(self, *args, **kwargs):
        self.child = args[0]

    def __init__(self, *args, **kwargs):
        self.init(*args, **kwargs)

    def set(self, **kwargs):
        for name, value in kwargs.items():
            try:
                attr = self.__getattribute__(name)
                self.__setattr__(name, value)
            except Exception as e:
                raise AttributeError(
                    f"Attribute '{name}' does not exist for tensor type {type(self)}"
                    )
        return self

    def __torch_function__(self, func, args=(), kwargs=None):
        print("runnuing")
        ""
        return execute_default_function_on_child_and_wrap(self, func, args,
            kwargs)

    def __add__(self, other):
        result = self.child + other.child
        wrapped_result = RestrictedTorchTensor(result.data)
        wrapped_result.child = result
        return wrapped_result


In [31]:
result = RestrictedTorchTensor(th.tensor([1,2,3,4.,], requires_grad=True))

x = RestrictedTorchTensor(result.data)
x.child = result

In [32]:
y = x + x

In [36]:
y.child.child += 1

In [66]:
x.backward(x)
x.grad

tensor([1., 2., 3., 4.])